In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error

In [2]:
# Step 1: Load the historical data
data = pd.read_csv('historical_data.csv')

In [3]:
# Step 2: Filter the required columns and sort by date and ticker
data = data[['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Ticker']]
data['Date'] = pd.to_datetime(data['Date'])
data.sort_values(['Ticker', 'Date'], inplace=True)

In [4]:
# Step 3: Encode the Ticker column
encoder = OneHotEncoder(sparse=False)
tickers_encoded = encoder.fit_transform(data[['Ticker']])

c:\Users\astev\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [5]:
# Step 4: Add the encoded tickers as additional features
encoded_columns = [f'Ticker_{i}' for i in range(tickers_encoded.shape[1])]
encoded_df = pd.DataFrame(tickers_encoded, columns=encoded_columns, index=data.index)
data = pd.concat([data.reset_index(drop=True), encoded_df], axis=1)

In [6]:
# Step 5: Scale numerical features (Open, High, Low, Close, Volume)
scaler = MinMaxScaler(feature_range=(0, 1))
numerical_features = ['Open', 'High', 'Low', 'Close', 'Volume']
data[numerical_features] = scaler.fit_transform(data[numerical_features])

In [7]:
# Step 6: Preprocessing function
def preprocess_data(data, n_steps=10):
    X, y = [], []
    feature_columns = numerical_features + encoded_columns
    for ticker in data['Ticker'].unique():
        ticker_data = data[data['Ticker'] == ticker] # Ensuring sequences are within the same stock
        ticker_data = ticker_data[feature_columns].values
        for i in range(n_steps, len(ticker_data)):
            X.append(ticker_data[i - n_steps:i, :])  # Last n_steps rows as features
            y.append(ticker_data[i, 3])  # Predict 'Close' price
    return np.array(X), np.array(y)

In [8]:
# Step 7: Preprocess the data
X, y = preprocess_data(data, n_steps=10)

In [9]:
# Step 8: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
print(X_train.shape)

(30105, 10, 103)


In [15]:
# Step 9: Build the LSTM model
model = Sequential([
    LSTM(50, activation="tanh", return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.15),
    LSTM(30, activation="tanh", return_sequences=True),
    Dropout(0.05),
    LSTM(20, activation="tanh", return_sequences=False),
    Dropout(0.01),
    Dense(1, activation="linear")
])

In [16]:
# Step 10: Compile the model
model.compile(optimizer='adam', loss='mse')

In [17]:
# Step 11: Train the model
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.1)

Epoch 1/20
847/847 ━━━━━━━━━━━━━━━━━━━━ 14s 9ms/step - loss: 9.1102e-04 - val_loss: 5.2892e-05
Epoch 2/20
847/847 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - loss: 1.1879e-04 - val_loss: 3.1937e-05
Epoch 3/20
847/847 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 7.8338e-05 - val_loss: 6.9376e-05
Epoch 4/20
847/847 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - loss: 8.1107e-05 - val_loss: 6.8381e-05
Epoch 5/20
847/847 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - loss: 9.5806e-05 - val_loss: 3.1687e-05
Epoch 6/20
847/847 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - loss: 7.4018e-05 - val_loss: 2.4504e-05
Epoch 7/20
847/847 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - loss: 7.0062e-05 - val_loss: 3.0914e-05
Epoch 8/20
847/847 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - loss: 6.4513e-05 - val_loss: 6.0543e-05
Epoch 9/20
847/847 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - loss: 6.0726e-05 - val_loss: 3.0102e-05
Epoch 10/20
847/847 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - loss: 5.2503e-05 - val_loss: 2.5798e-05
Epoch 11/20
847/847 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - l

In [18]:
# Step 12: Make predictions
predictions = model.predict(X_test)

236/236 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step


In [19]:
# Step 13: Inverse scale the predictions and actual values
y_test_scaled = scaler.inverse_transform(
    np.concatenate((np.zeros((y_test.shape[0], 4)), y_test.reshape(-1, 1)), axis=1))[:, -1]
predictions_scaled = scaler.inverse_transform(
    np.concatenate((np.zeros((predictions.shape[0], 4)), predictions), axis=1))[:, -1]

In [20]:
# Step 13: Evaluate the model
mae = mean_absolute_error(predictions_scaled, y_test_scaled)
mape = mean_absolute_percentage_error(predictions_scaled, y_test_scaled)

# Define a percentage threshold for accuracy
threshold_percentage = 5  # 5% tolerance

# Calculate percentage errors
percentage_errors = np.abs((y_test_scaled - predictions_scaled) / y_test_scaled) * 100

# Count predictions within the threshold
acc = np.mean(percentage_errors <= threshold_percentage) * 100

acc2 = 1- mape


In [21]:
# Step 14: Print performance metrics
print(f"Mean Absolute Error = {mae}")
print(f"Mean Absolute Percentage Error = {mape*100:.2f}%")
print(f"Accuracy = {acc:.2f}%")
print(f"Accuracy = {acc2 *100:.2f}%")

Mean Absolute Error = 3876087.8863592604
Mean Absolute Percentage Error = 7.31%
Accuracy = 55.61%
Accuracy = 92.69%
